In [1]:
import pandas as pd
import numpy as np
import cyvcf2

In [2]:
mydata = "../ValidationData/mydata/original.snpeff.state.disease.identifiedgene.filtered.splai.tsv"
df = pd.read_table(mydata, sep='\t', dtype=str)

In [3]:
from cyvcf2 import Writer

In [ ]:
out_vcf = 